In [48]:
import numpy as np
import pandas as pd
import requests
import json
import feather
import os
from time import sleep

In [49]:
api_key = ''

steam_id_url = 'http://api.steampowered.com/ISteamApps/GetAppList/v2'
steam_details_api = 'http://store.steampowered.com/api/appdetails/?appids='
ratings_api = 'http://steamspy.com/api.php?request=appdetails&appid='
app_path = '/Users/nmouterde/Documents/projectsPython/steam-game-engine/'

In [55]:
steam_ids = requests.get(steam_id_url)

In [56]:
steam_ids = json.loads(steam_ids.text)

In [57]:
df_steam_ids = pd.DataFrame.from_dict(steam_ids['applist']['apps'])

In [ ]:
games = []
i=0

In [187]:
i=0

In [191]:
for id_app in df_steam_ids.appid.values[12400:]:
    if i%500==0:
        print i
    i=i+1
    api_call = requests.get(steam_details_api + str(id_app))
    rates_call = requests.get(ratings_api + str(id_app))
    rates = json.loads(rates_call.text)
    app_desc = json.loads(api_call.text)
    game = app_desc[app_desc.keys()[0]]
    if game['success']:
        game_info = game['data']
        if game_info['type'] == "game":
            if 'categories' in game_info:
                cats = [cat['description'] for cat in game_info['categories']]
                cats = ' '.join(cats)
            else:
                cats = ' '
            genres = [genre['description'] for genre in game_info['genres']]
            genres = ' '.join(genres)

            if 'developers' in game_info:
                new_game = [game_info['steam_appid'], game_info['name'],
                              game_info['detailed_description'],
                              game_info['short_description'], 
                              game_info['about_the_game'],
                              game_info['developers'][0],
                              game_info['publishers'][0],
                              cats, genres,
                              rates['score_rank']]
            else:
                new_game = [game_info['steam_appid'], game_info['name'],
                              game_info['detailed_description'],
                              game_info['short_description'], 
                              game_info['about_the_game'],
                              ' ',
                              game_info['publishers'][0],
                              cats, genres,
                              rates['score_rank']]
            
            if 'recommendations' not in game_info:
                new_game.append(0)
            else:
                new_game.append(game_info['recommendations']['total'])
            
            games.append(new_game)
            
df_cols = ['id', 'name',
           'f_desc', 's_desc','about',
           'dev' , 'pub', 'cat',
           'genres', 'score', 'nb_rec']

df_info = pd.DataFrame.from_records(games, columns=df_cols)       
feather.write_dataframe(df_info, os.path.join(app_path + 'data_processed/scrap_df.f'))

AttributeError: 'NoneType' object has no attribute 'keys'

In [151]:
api_call.text
steam_details_api + str(id_app)

'http://store.steampowered.com/api/appdetails/?appids=258993'

In [189]:
i

202

In [190]:
len(games)

1376

In [107]:
df_steam_ids.appid.values[2500:2510]

array([24078, 24079, 24080, 24081, 24082, 24083, 24084, 24085, 24086, 24087])